### 第一题

使用朴素贝叶斯过滤垃圾邮件

#### 题目



(a). 收集数据：提供文本文件。

(b). 准备数据：将文本文件解析成词条向量。

(c). 分析数据：检查词条确保解析的正确性。

(d). 训练算法：使用我们之前建立的trainNB0() 函数。

(e). 测试算法：使用classifyNB()，并且构建一个新的测试函数来计算文档集的错误率。

(f). 使用算法：构建一个完整的程序对一组文档进行分类，将错分的文档输出到屏幕上。

#### 解答

##### 收集数据

读取所有文本，构建单词表

In [1]:
import re# 这个库中包含了正则表达式的函数
import numpy as np
# 定义一个函数，用于将文本转换为词列表表
def text_to_word_list(text):
  # 使用正则表达式将文本中的所有非字母数字字符替换为空格
  text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
  # 将文本转换为小写
  text = text.lower()
  # 使用 split() 函数将文本分割成单词列表
  word_list = text.split()
  # 返回单词列表
  return word_list

def createMailList(path):
    mail_list=[]
    for i in range(1,26):
        mail_list.append(text_to_word_list(open(path+"/ham/"+str(i)+".txt",'r',encoding='gbk').read()))
        #print(open(route,'r',encoding='gbk').read())
    for i in range(1,26):
        mail_list.append(text_to_word_list(open(path+"/spam/"+str(i)+".txt",'r',encoding='gbk').read()))
        #print(open(route,'r',encoding='gbk').read())
    return mail_list

def createVocabList(mail_list):
    vocabSet = set([]) #创建一个空的不重复列表
    for words_list in mail_list:
        vocabSet = vocabSet | set(words_list) #取并集
    return list(vocabSet)

path="./email"
mail_list=createMailList(path)
vocab_list=createVocabList(mail_list)
class_vec=np.concatenate((np.zeros(25,dtype=int),np.ones(25,dtype=int))).tolist()

#print(vocab_list)
#print(class_vec)

##### 准备数据

将文本文件解析成词条向量

In [2]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList) #创建一个其中所含元素都为0的向量
    for word in inputSet: #遍历每个词条
        if word in vocabList: #如果词条存在于词汇表中，则置1
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec #返回文档向量

##### 分析数据

检查词条确保解析的正确性。

In [3]:
print("字典列表：")
print(vocab_list)
print("\n\n")
temp=setOfWords2Vec(vocab_list,["this","is","a","test","text"])

字典列表：
['vicodin', 'hope', 'to', 'china', 'quality', 'magazine', '130', 'fractal', 'penisen1argement', 'died', '180', 'and', 'products', 'keep', 'questions', 'foaming', 'please', 'gain', 'jqplot', 'plugin', 'reservation', 'courier', 'suggest', 'py', 'model', 'm', 'right', 'over', 'supporting', 'stuff', 'how', 'hermes', '2010', 'windows', 'length', 'accepted', 'need', 'mailing', '20', 'xp', 'viagranoprescription', 'photoshop', 'only', 'enjoy', 'files', '625', 'said', 'serial', 'discussions', '322', 'that', 'saw', 'natural', 'accept', 'analgesic', 'cold', '366', 'winter', 'strategic', 'may', 'blue', 'google', 'buyviagra', 'made', 'york', 'delivery', 'talked', 'linkedin', 'number', 'style', 'by', 'find', 'pro', 'mandarin', 'store', 'much', 'doctor', 'prices', '513', 'louis', 'germany', 'mba', 'chinese', 'launch', 'expo', '100mg', 'release', 'speedpost', 'ferguson', 'watches', '39', 'tv', 'haloney', 'working', 'lists', 'or', 'assigning', 'zolpidem', 'derivatives', 'such', 'ideas', 'thirumal

##### 训练算法

使用我们之前建立的trainNB0()函数

In [4]:

def trainNB0(trainMatrix,trainCategory):
    '''
    Parameters:

    trainMatrix - 训练文档矩阵，即setOfWords2Vec返回的returnVec构成的矩阵

    trainCategory - 训练类别标签向量，即loadDataSet返回的classVec

    Returns:

    p0Vect - 侮辱类的条件概率数组

    p1Vect - 非侮辱类的条件概率数组

    pAbusive - 文档属于侮辱类的概率

    p0Vect

    = p(w|文档属于非侮辱类)

    = [p(w0|文档属于非侮辱类),p(w1|文档属于非侮辱类),.....]

    = 文档属于非侮辱类的情况下：[第0个单词的出现频率，第1个单词的出现频率，.....]

    = 文档属于非侮辱类的情况下：[第0个单词的出现次数，第1个单词的出现次数，.....] / 非侮辱类文档单词总数

    = p0Num / p0Denom
    '''
    # 计算训练的文档数目
    numTrainDocs = len(trainMatrix)
    # 计算每篇文档的词条数
    numWords = len(trainMatrix[0])
    # 文档属于侮辱类的概率
    pAbusive = sum(trainCategory)/float(numTrainDocs)

    
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)
    #p1Num是分子列表，其长度为单词表的长度，[]
    p0Denom = 2.0
    p1Denom = 2.0 #分母初始化为2,拉普拉斯平滑

    for i in range(numTrainDocs):
    # 统计属于垃圾邮件的条件概率所需的数据
        if trainCategory[i] == 1: # 如果trainMatrix[i]是垃圾邮件
            p1Num += trainMatrix[i] 
            p1Denom += sum(trainMatrix[i])
        else:# 统计属于正常邮件的条件概率所需的数据
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # 取对数，防止下溢出
    p1Vect = np.log(p1Num/p1Denom)
    p0Vect = np.log(p0Num/p0Denom)
    # 返回属于正常邮件的条件概率数组，属于垃圾邮件的条件概率数组，文档属于垃圾邮件的概率
    return p0Vect,p1Vect,pAbusive


trainMat = []
for postinDoc in mail_list:
    trainMat.append(setOfWords2Vec(vocab_list, postinDoc))
p0V, p1V, pAb = trainNB0(trainMat, class_vec)
print('p0V:\n', p0V)
print('p1V:\n', p1V)
print('classVec:\n', class_vec)
print('pAb:\n', pAb)


p0V:
 [-6.82546004 -6.13231286 -3.88102106 -6.13231286 -6.82546004 -6.13231286
 -6.82546004 -6.13231286 -6.82546004 -6.13231286 -6.82546004 -4.05287131
 -6.13231286 -6.13231286 -6.13231286 -6.13231286 -6.13231286 -6.82546004
 -6.13231286 -6.13231286 -6.13231286 -6.82546004 -6.13231286 -6.13231286
 -6.13231286 -5.72684775 -5.72684775 -6.82546004 -6.13231286 -5.72684775
 -5.72684775 -6.82546004 -6.13231286 -6.13231286 -6.82546004 -6.82546004
 -6.13231286 -6.13231286 -6.13231286 -6.82546004 -6.82546004 -6.82546004
 -5.43916568 -6.13231286 -6.13231286 -6.82546004 -6.13231286 -6.13231286
 -6.13231286 -6.82546004 -4.74601849 -6.13231286 -6.82546004 -5.72684775
 -6.82546004 -5.72684775 -6.82546004 -6.13231286 -6.13231286 -5.72684775
 -6.13231286 -6.13231286 -6.82546004 -6.13231286 -6.13231286 -6.82546004
 -6.13231286 -5.72684775 -6.13231286 -6.13231286 -5.43916568 -6.82546004
 -6.82546004 -6.13231286 -5.72684775 -6.13231286 -6.82546004 -6.13231286
 -6.82546004 -6.82546004 -6.13231286 -6.13231

##### 测试算法

使用classifyNB()，并且构建一个新的测试函数来计算文档集的错误率。

In [5]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    '''
    比较P(class=0 | w=vec2Classify )与P(class=1 | w=vec2Classify )的大小

    因为对于class=1和class=0的P(ci|w)=p(w|ci)*p(ci)/p(w)中，p(w)不变，因此只比较分子

    因为p(w|ci)是经过取对数的，因此对p(ci)也取对数
    '''

    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1) 
    #对应元素相乘。logA * B = logA + logB，
    #所以这里加上log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0


##### 使用算法

构建一个完整的程序对一组文档进行分类，将错分的文档输出到屏幕上。

将本文档的代码块合起来并去除多余的输出语句即为完整的程序

In [6]:
trainingSet = list(range(50)) #trainingSet是记录用作训练数据的单词列表在mail_list中的下标
testSet=[]#testSet是记录用作测试数据的单词列表的在mail_list中的下标
for i in range(16):# 随机抽取16个邮件作为训练集
    randIndex = int(np.random.uniform(0,len(trainingSet)))
    testSet.append(trainingSet[randIndex])
    del trainingSet[randIndex]
trainMat=[]
trainClasses = []
for docIndex in trainingSet:
    trainMat.append(setOfWords2Vec(vocab_list, mail_list[docIndex]))
    trainClasses.append(class_vec[docIndex])
p0V,p1V,pSpam = trainNB0(np.array(trainMat),np.array(trainClasses))
# 错误数
errorCount = 0
for docIndex in testSet:
    wordVector = setOfWords2Vec(vocab_list, mail_list[docIndex])
    if classifyNB(np.array(wordVector),p0V,p1V,pSpam) != class_vec[docIndex]:
        errorCount += 1
        print ("分类错误的邮件：\n", mail_list[docIndex])
        print(f"应为{'垃圾邮件' if class_vec[docIndex]==1 else '非垃圾邮件' }，被错误分类为{'垃圾邮件' if class_vec[docIndex]==0 else '非垃圾邮件' }")

print ('the error rate is: ', float(errorCount)/len(testSet))

the error rate is:  0.0
